In [1]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest,MinHashLSH
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data=pd.read_csv("C:/Aditya/DM_project/rows.csv")

C:\Users\adity\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4,5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.shape

(1282355, 18)

In [4]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228
2,05/10/2019,Debt collection,Payday loan debt,Communication tactics,Frequent or repeated calls,NaN,NaN,CURO Intermediate Holdings,TX,751XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237964
3,05/10/2019,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Old information reappears or never goes away,NaN,NaN,Ad Astra Recovery Services Inc,LA,708XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3238479
4,05/10/2019,Checking or savings account,Checking account,Managing an account,Banking errors,NaN,NaN,ALLY FINANCIAL INC.,AZ,85205,NaN,NaN,Postal mail,05/10/2019,In progress,Yes,NaN,3238460


## Exploratory data analysis

In [5]:
data_for_eda=data.copy()
data_for_eda["freq_of_occur"]=1

In [6]:
data_for_eda.shape

(1282355, 19)

In [7]:
data_for_eda_not_null_=data_for_eda.loc[data_for_eda['Consumer complaint narrative'].notnull()].copy()
data_for_eda_not_null =data_for_eda_not_null_.groupby("Issue").filter(lambda x: len(x) > 10000)

In [8]:
data_for_eda_not_null.shape

(184349, 19)

In [9]:
data_for_eda_not_null['Issue'].value_counts(normalize=True)

Incorrect information on your report                                                0.264569
Problem with a credit reporting company's investigation into an existing problem    0.134131
Incorrect information on credit report                                              0.115091
Attempts to collect debt not owed                                                   0.107649
Cont'd attempts collect debt not owed                                               0.094576
Improper use of your report                                                         0.083179
Loan servicing, payments, escrow account                                            0.079859
Communication tactics                                                               0.062414
Loan modification,collection,foreclosure                                            0.058530
Name: Issue, dtype: float64

In [10]:
Issue_across_pdt=data_for_eda_not_null.groupby(["Issue","Product"],as_index=False).agg({"freq_of_occur":'count'})
Issue_across_pdt.sort_values("Issue")
Issue_across_pdt.head(100)

,Issue,Product,freq_of_occur
0,Attempts to collect debt not owed,Debt collection,19845
1,Communication tactics,Debt collection,11506
2,Cont'd attempts collect debt not owed,Debt collection,17435
3,Improper use of your report,Checking or savings account,13
4,Improper use of your report,Credit card or prepaid card,86
5,Improper use of your report,"Credit reporting, credit repair services, or other personal consumer reports",15085
6,Improper use of your report,Mortgage,33
7,Improper use of your report,"Payday loan, title loan, or personal loan",48
8,Improper use of your report,Student loan,19
9,Improper use of your report,Vehicle loan or lease,50


In [11]:
Issue_across_pdt=data_for_eda_not_null.groupby(["Issue","Product"],as_index=False).agg({"freq_of_occur":'count'})
Issue_across_pdt.groupby(["Issue"],as_index=False).agg({"Product":'count'})

,Issue,Product
0,Attempts to collect debt not owed,1
1,Communication tactics,1
2,Cont'd attempts collect debt not owed,1
3,Improper use of your report,7
4,Incorrect information on credit report,1
5,Incorrect information on your report,7
6,"Loan modification,collection,foreclosure",1
7,"Loan servicing, payments, escrow account",1
8,Problem with a credit reporting company's investigation into an existing problem,7


### Keeping only those columns that are important for our analysis

In [12]:
cols_for_focus=['Product','Sub-product','Issue','Sub-issue','Consumer complaint narrative']
data_t1=data[cols_for_focus].copy()
data_t2=data_t1.loc[data_t1['Consumer complaint narrative'].notnull()].copy()

data_t2['dummy']=1
data_t2.shape


(383564, 6)

### Filtering and keeping only those Issues that occur more than 10000 times 

In [13]:
for_analysis_t2 =data_t2.groupby("Issue").filter(lambda x: len(x) > 10000)

In [14]:
for_analysis_t2['Issue'].value_counts(normalize=True)

Incorrect information on your report                                                0.264569
Problem with a credit reporting company's investigation into an existing problem    0.134131
Incorrect information on credit report                                              0.115091
Attempts to collect debt not owed                                                   0.107649
Cont'd attempts collect debt not owed                                               0.094576
Improper use of your report                                                         0.083179
Loan servicing, payments, escrow account                                            0.079859
Communication tactics                                                               0.062414
Loan modification,collection,foreclosure                                            0.058530
Name: Issue, dtype: float64

In [15]:
### Data cleaning step -- Commented as final dataset is created below and read in for analysis
'''
for_analysis_t2['Consumer complaint narrative']=for_analysis_t2['Consumer complaint narrative'].str.lower()
lemma = WordNetLemmatizer()
to_remove = stopwords.words("english")
def remove_non_string(x):
    
    t1=re.sub('[^A-Za-z\s]+', '',x)
    t2=re.sub('x{2,20}', '',t1)
    return (t2)

#https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return (wordnet.ADJ)
    elif nltk_tag.startswith('V'):
        return (wordnet.VERB)
    elif nltk_tag.startswith('N'):
        return (wordnet.NOUN)
    elif nltk_tag.startswith('R'):
        return (wordnet.ADV)
    else:
        return (None)
    
def lem_word(x):
    word_and_postag = nltk.pos_tag(nltk.word_tokenize(x))
    word_tag=list(map(lambda x: (x[0], pos_tagger(x[1])), word_and_postag))
    lemmatized_sentence = []
    for word, tag in word_tag:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemma.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return(lemmatized_sentence)

def rem_stop(x):
        y = ' '.join([sw for sw in x.split() if sw not in to_remove])
        return(y)

for_analysis_t2['Consumer complaint narrative_']=for_analysis_t2['Consumer complaint narrative'].progress_apply(remove_non_string)
for_analysis_t2['Consumer complaint narrative_']=for_analysis_t2['Consumer complaint narrative_'].progress_apply(lem_word)
for_analysis_t2['Consumer complaint narrative_']=for_analysis_t2['Consumer complaint narrative_'].progress_apply(rem_stop)

# test="the cat is sitting with the bats on the striped mat under many badly flying geese"
# print(lem_word(test))
#re.sub('[^A-Za-z0-9]+', '', mystring)
'''


'\nfor_analysis_t2[\'Consumer complaint narrative\']=for_analysis_t2[\'Consumer complaint narrative\'].str.lower()\nlemma = WordNetLemmatizer()\nto_remove = stopwords.words("english")\ndef remove_non_string(x):\n    \n    t1=re.sub(\'[^A-Za-z\\s]+\', \'\',x)\n    t2=re.sub(\'x{2,20}\', \'\',t1)\n    return (t2)\n\n#https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/\ndef pos_tagger(nltk_tag):\n    if nltk_tag.startswith(\'J\'):\n        return (wordnet.ADJ)\n    elif nltk_tag.startswith(\'V\'):\n        return (wordnet.VERB)\n    elif nltk_tag.startswith(\'N\'):\n        return (wordnet.NOUN)\n    elif nltk_tag.startswith(\'R\'):\n        return (wordnet.ADV)\n    else:\n        return (None)\n    \ndef lem_word(x):\n    word_and_postag = nltk.pos_tag(nltk.word_tokenize(x))\n    word_tag=list(map(lambda x: (x[0], pos_tagger(x[1])), word_and_postag))\n    lemmatized_sentence = []\n    for word, tag in word_tag:\n        if tag is None:\n            # if there is

### Reading in the file for the analysis

In [16]:
# for_analysis_t2.to_csv('C:/Aditya/DM_project/cleaned_data.csv')

In [17]:
for_analysis_t2 = pd.read_csv('C:/Aditya/DM_project/cleaned_data.csv')
del for_analysis_t2['Unnamed: 0']
del for_analysis_t2['dummy']

for_analysis_t2['dummy']=1

In [18]:
for_analysis_t2['Issue'].value_counts(normalize=True)

Incorrect information on your report                                                0.264569
Problem with a credit reporting company's investigation into an existing problem    0.134131
Incorrect information on credit report                                              0.115091
Attempts to collect debt not owed                                                   0.107649
Cont'd attempts collect debt not owed                                               0.094576
Improper use of your report                                                         0.083179
Loan servicing, payments, escrow account                                            0.079859
Communication tactics                                                               0.062414
Loan modification,collection,foreclosure                                            0.058530
Name: Issue, dtype: float64

### Splitting the data so that can train and test

In [19]:
from sklearn.model_selection import train_test_split
X = for_analysis_t2.iloc[:, :-1].values
Y = for_analysis_t2.iloc[:, -1].values
# print(X[:5])
# print(Y[:5])

In [20]:
X_train, X_test,y_train,y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [21]:
for_analysis_t2.columns

Index(['Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Consumer complaint narrative_',
       'dummy'],
      dtype='object')

### We have run the minhash algorithm based on the below links
http://ekzhu.com/datasketch/lshforest.html

https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/

https://www.fatalerrors.org/a/text-similarity-calculation-minhash-and-lsh-algorithm.html

In [22]:
def get_forest(data, perms):
        
    minhash = []
    
    for text in data['Consumer complaint narrative_']:
        indi_words = text.split()
        m = MinHash(num_perm=perms)
        for s in indi_words:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
        
    return forest


In [23]:
#Number of Permutations
permutations = 128


In [24]:
col_names=['Product', 'Sub-product', 'Issue', 'Sub-issue','Consumer complaint narrative', 'Consumer complaint narrative_']
X_train_df=pd.DataFrame(X_train)
X_train_df.columns=col_names
# print(X_train_df.head())
# print("-"*50)
X_test_df=pd.DataFrame(X_test)
X_test_df.columns=col_names
# print(X_test_df.head())

In [25]:
X_train_df['len_of_string']=X_train_df['Consumer complaint narrative_'].str.len()
X_train_df.describe()

,len_of_string
count,123510.000000
mean,499.406323
std,551.829216
min,2.000000
25%,173.000000
50%,330.000000
75%,623.000000
max,19065.000000


In [26]:

def predict(text, database, perms, num_results, forest):
    
    #Splitting the complaint data into individual words
    indi_words = text.split()

    #Initializing the minhash with the permutations
    m = MinHash(num_perm=perms)

    for s in indi_words:
        m.update(s.encode('utf8'))

    #Finding the match from the minhash forest    
    matches = np.array(forest.query(m, num_results))

    #It is possible that the minhash might not find any matches
    if len(matches) == 0:
        return None 
    
    result = database.loc[matches]['Issue']

    
    return result

In [27]:
X_train_df_fin = X_train_df[X_train_df['Consumer complaint narrative_'].notnull()]
X_train_df_fin.shape


(123510, 7)

In [28]:
forest = get_forest(X_train_df_fin, permutations)

In [29]:
X_test_df_fin = X_test_df[X_test_df['Consumer complaint narrative_'].notnull()]
X_test_df_fin.shape

(60831, 6)

In [30]:
import operator

def sorted_dict(x):
    list_x=list(x)
    issue_dist={}
    for issue in list_x:
        if issue_dist.get(issue,None) == None:
            issue_dist.update({issue:1})
        else:
            issue_dist[issue]+=1

    best_match={}
    for k,l in issue_dist.items():
        tot=sum(issue_dist.values())
        prob=float(l/tot)
        if best_match.get(k,None) == None:
            best_match.update({k:prob})
    sorted_d = dict( sorted(best_match.items(), key=operator.itemgetter(1),reverse=True))
            
    return(sorted_d)


### Approach 1: Based on best recommendation

In [31]:
actual_prd=[]
row_to_test=126577-1
num_recommendations = 50

for i in range(0,X_test.shape[0]-1):
   
    to_find_out = X_test[i][5]
    tag=X_test[i][2]
    
    try:
        result = predict(to_find_out, X_train_df, permutations, num_recommendations, forest)
    except:
        print("")
    else:
        result = predict(to_find_out, X_train_df, permutations, num_recommendations, forest)

    try:
        out=sorted_dict(result)
    except:
        actual_prd.append([tag,"no match"])
    else:
        actual_prd.append([tag,list(out.keys())[0]])
    


### Creating a dataframe to have the actual and predicted issues

In [32]:
fin_df=pd.DataFrame(actual_prd)
fin_names=['actual','predicted']
fin_df.columns=fin_names
fin_df.head()

,actual,predicted
0,"Loan modification,collection,foreclosure",Incorrect information on your report
1,Problem with a credit reporting company's investigation into an existing problem,Incorrect information on your report
2,Attempts to collect debt not owed,Incorrect information on your report
3,"Loan servicing, payments, escrow account",Incorrect information on your report
4,Incorrect information on your report,Incorrect information on your report


In [33]:
def flag_row(x):
    if x['actual'] == x['predicted']:
        val = 1
    else:
        val = 0
    return val


fin_df['correct']=fin_df.apply(flag_row,axis=1)
fin_df.head()


,actual,predicted,correct
0,"Loan modification,collection,foreclosure",Incorrect information on your report,0
1,Problem with a credit reporting company's investigation into an existing problem,Incorrect information on your report,0
2,Attempts to collect debt not owed,Incorrect information on your report,0
3,"Loan servicing, payments, escrow account",Incorrect information on your report,0
4,Incorrect information on your report,Incorrect information on your report,1


In [34]:
#class_by_cat=fin_df.groupby(['actual','correct'],as_index='False').agg({'actual':'count'})

class_by_cat=fin_df.pivot_table(index="actual", columns="correct", aggfunc="count",fill_value=0)
class_by_cat.head()

predicted     
correct                                        0    1
actual                                               
Attempts to collect debt not owed           6012  578
Communication tactics                       3386  380
Cont'd attempts collect debt not owed       5419  405
Improper use of your report                 4341  688
Incorrect information on credit report      6655  401

In [35]:
class_by_cat.columns = ['predicted_0','predicted_1']

### We can clearly see that for the issues that occur the highest in terms of proportion in the data the accuracy is 87%

In [36]:
class_by_cat.head()
class_by_cat['total']=class_by_cat['predicted_0']+class_by_cat['predicted_1']
class_by_cat['Percent_correct']=class_by_cat['predicted_1']/class_by_cat['total']
class_by_cat['Occurence_of_issue']=100*(class_by_cat['total']/class_by_cat['total'].sum())
class_by_cat.sort_values(by=['total'],ascending=False,inplace=True)

class_by_cat['cumulative_issue_percentage']=100*(class_by_cat['total'].cumsum()/class_by_cat['total'].sum())
class_by_cat.head(50)

,predicted_0,predicted_1,total,Percent_correct,Occurence_of_issue,cumulative_issue_percentage
actual,,,,,,
Incorrect information on your report,2102,13912,16014,0.868740,26.323662,26.323662
Problem with a credit reporting company's investigation into an existing problem,6707,1381,8088,0.170747,13.294978,39.618641
Incorrect information on credit report,6655,401,7056,0.056831,11.598586,51.217227
Attempts to collect debt not owed,6012,578,6590,0.087709,10.832580,62.049807
Cont'd attempts collect debt not owed,5419,405,5824,0.069540,9.573436,71.623243
Improper use of your report,4341,688,5029,0.136807,8.266623,79.889866
"Loan servicing, payments, escrow account",4410,396,4806,0.082397,7.900058,87.789924
Communication tactics,3386,380,3766,0.100903,6.190515,93.980439
"Loan modification,collection,foreclosure",3379,283,3662,0.077280,6.019561,100.000000


In [37]:
class_by_cat['predicted_1'].sum()/(class_by_cat['predicted_1'].sum()+class_by_cat['predicted_0'].sum())


0.30285197665817376

## Approach 2: Taking top 2 recommended issues based on the proportion

In [38]:
actual_prd_top3=[]
row_to_test=126577-1
num_recommendations = 50

for i in range(0,X_test.shape[0]-1):
   
    to_find_out = X_test[i][5]
    tag=X_test[i][2]
    
    try:
        result = predict(to_find_out, X_train_df, permutations, num_recommendations, forest)
    except:
        print("")
    else:
        result = predict(to_find_out, X_train_df, permutations, num_recommendations, forest)

    try:
        out=sorted_dict(result)
    except:
        actual_prd_top3.append([tag,"no match"])
    else:
        actual_prd_top3.append([tag,list(out.keys())[0:2]])

### Creating a dataframe to have the actual and predicted issues

In [39]:
fin_df_top3=pd.DataFrame(actual_prd_top3)
fin_names=['actual','predicted']
fin_df_top3.columns=fin_names
fin_df_top3.head()

,actual,predicted
0,"Loan modification,collection,foreclosure","[Incorrect information on your report, Incorrect information on credit report]"
1,Problem with a credit reporting company's investigation into an existing problem,"[Incorrect information on your report, Communication tactics]"
2,Attempts to collect debt not owed,"[Incorrect information on your report, Incorrect information on credit report]"
3,"Loan servicing, payments, escrow account","[Incorrect information on your report, Attempts to collect debt not owed]"
4,Incorrect information on your report,"[Incorrect information on your report, Incorrect information on credit report]"


In [40]:
def flag_row(x):
    
    y=x['predicted']
    val=0
    for i in y:
        if x['actual'] == i:
            val = 1
            break
        else:
            val = 0
    return val


fin_df_top3['correct']=fin_df_top3.apply(flag_row,axis=1)
fin_df_top3.head()


,actual,predicted,correct
0,"Loan modification,collection,foreclosure","[Incorrect information on your report, Incorrect information on credit report]",0
1,Problem with a credit reporting company's investigation into an existing problem,"[Incorrect information on your report, Communication tactics]",0
2,Attempts to collect debt not owed,"[Incorrect information on your report, Incorrect information on credit report]",0
3,"Loan servicing, payments, escrow account","[Incorrect information on your report, Attempts to collect debt not owed]",0
4,Incorrect information on your report,"[Incorrect information on your report, Incorrect information on credit report]",1


In [41]:
#class_by_cat=fin_df.groupby(['actual','correct'],as_index='False').agg({'actual':'count'})

class_by_cat=fin_df_top3.pivot_table(index="actual", columns="correct", aggfunc="count",fill_value=0)
class_by_cat

predicted  \
correct                                                                                  0   
actual                                                                                       
Attempts to collect debt not owed                                                     4548   
Communication tactics                                                                 2848   
Cont'd attempts collect debt not owed                                                 4517   
Improper use of your report                                                           3386   
Incorrect information on credit report                                                4854   
Incorrect information on your report                                                   555   
Loan modification,collection,foreclosure                                              2877   
Loan servicing, payments, escrow account                                              3390   
Problem with a credit reporting company's investigation into an existing problem      3496   

                                                                                         
correct                                                                               1  
actual                                                                                   
Attempts to collect debt not owed                                                  2042  
Communication tactics                                                               918  
Cont'd attempts collect debt not owed                                              1307  
Improper use of your report                                                        1643  
Incorrect information on credit report                                             2202  
Incorrect information on your report                                              15459  
Loan modification,collection,foreclosure                                            785  
Loan servicing, payments, escrow account                                           1416  
Problem with a credit reporting company's investigation into an existing problem   4592

In [42]:
class_by_cat.columns = ['predicted_0','predicted_1']

### We can clearly see that the prediction has significantly improved across issue types and also at an overall level

In [43]:
class_by_cat['total']=class_by_cat['predicted_0']+class_by_cat['predicted_1']
class_by_cat['Percent_correct']=class_by_cat['predicted_1']/class_by_cat['total']
class_by_cat['Occurence_of_issue']=100*(class_by_cat['total']/class_by_cat['total'].sum())
class_by_cat.sort_values(by=['total'],ascending=False,inplace=True)

class_by_cat['cumulative_issue_percentage']=100*(class_by_cat['total'].cumsum()/class_by_cat['total'].sum())
class_by_cat.head(50)

,predicted_0,predicted_1,total,Percent_correct,Occurence_of_issue,cumulative_issue_percentage
actual,,,,,,
Incorrect information on your report,555,15459,16014,0.965343,26.323662,26.323662
Problem with a credit reporting company's investigation into an existing problem,3496,4592,8088,0.567755,13.294978,39.618641
Incorrect information on credit report,4854,2202,7056,0.312075,11.598586,51.217227
Attempts to collect debt not owed,4548,2042,6590,0.309863,10.832580,62.049807
Cont'd attempts collect debt not owed,4517,1307,5824,0.224416,9.573436,71.623243
Improper use of your report,3386,1643,5029,0.326705,8.266623,79.889866
"Loan servicing, payments, escrow account",3390,1416,4806,0.294632,7.900058,87.789924
Communication tactics,2848,918,3766,0.243760,6.190515,93.980439
"Loan modification,collection,foreclosure",2877,785,3662,0.214364,6.019561,100.000000


In [44]:
class_by_cat['predicted_1'].sum()/(class_by_cat['predicted_1'].sum()+class_by_cat['predicted_0'].sum())


0.4991205720391222